In [1]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

In [ ]:
purging the data model entirely - empty the schema

In [ ]:
if (haskey(ENV, "GENIE_ENV") && ENV["GENIE_ENV"] == "dev")
    if (haskey(ENV, "GITPOD_REPO_ROOT"))
        run(```psql -f sqlsnippets/droptables.sql```)
    else
        println("execute this in Your REPL")
        println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)""")
    end
end

Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [2]:
LifeInsuranceDataModel.load_model()

cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:305


┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsw_validfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ (3), is_committed BIGINT )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE testtstzranges (id BIGSERIAL PRIMARY KEY, i TSTZRANGE )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: CREATE TABLE validityIntervals (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsworld_validfrom TIMESTAMPTZ (3), tsworld_invalidfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ (3), tsdb_invalidfrom TIMESTAMPTZ (3), tsrworld TSTZRANGE , tsrdb TSTZRANGE , is_committed BIGINT )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE testdummyComponents (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE TABLE testdummyComponentRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES testdummyComponents(id), ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCA

┌ Info: CREATE TABLE testdummySubComponentRevisions (id BIGSERIAL PRIMARY KEY, ref_component BIGINT REFERENCES testdummySubComponents(id), ref_validfrom BIGINT REFERENCES versions(id) ON DELETE CASCADE, ref_invalidfrom BIGINT DEFAULT 9223372036854775807 REFERENCES versions(id) ON DELETE SET DEFAULT, ref_valid INT8RANGE , description VARCHAR )
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE EXTENSION IF NOT EXISTS btree_gist;
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  extension "btree_gist" already exists, skipping
┌ Info: ALTER TABLE validityIntervals 
│ ADD CONSTRAINT bitemp EXCLUDE USING GIST (ref_version WITH =, is_committed WITH =, tsrworld WITH &&, tsrdb WITH &&)
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: CREATE OR REPLACE FUNCTION f_bitempranges ()
│

...

┌ Info: INSERT INTO tariffitemroles ( "domain", "value" ) VALUES ( E'TariffItem', E'Supplementary Coverage - Occupational Disablity' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" WHERE "id" = 2 ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO tariffitemroles ( "domain", "value" ) VALUES ( E'TariffItem', E'Supplementary Coverage - Terminal Illness' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemro

┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Main Coverage - Life' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" WHERE "id" = 1 ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Supplementary Coverage - Occupational Disablity' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" WHERE "id" = 2 ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO productpartroles ( "domain", "value" ) VALUES ( E'ProductPart', E'Supplementary Coverage - Terminal Illness' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain

┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "productpartroles"."id" AS "productpartroles_id", "productpartroles"."domain" AS "productpartroles_domain", "productpartroles"."value" AS "productpartroles_value" FROM "productpartroles" ORDER BY productpartroles.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


4-element Vector{Int64}:
 1
 2
 3
 4

Create a Partner

In [3]:
# create Partner
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'1', E'1', E'2038-01-19T03:14:06.999+00:00', E'2014-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 1 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: INSERT INTO validityintervals ( "ref_history", "ref_version", "tsdb_validfrom", "tsworld_validfrom", "tsdb_invalidfrom", "tsworld_invalidfrom", "is_committed" ) VALUES ( E'1', E'1', E'2022-07-07T21:15:16.085+01:00', E'2014-05-30T20:00:01.001+00:00', E'2038-01-19T03:14:06.999+00:00', E'2038-01-19T03:14:06.999+00:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (1,1,1,"2014-05-30 20:00:01.001+00","2038-01-19 03:14:06.999+00","2022-07-07 20:15:16.085+00","2038-01-19 03:14:06.999+00",,,0)
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom"

...

┌ Info: INSERT INTO partnerrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description" ) VALUES ( E'1', E'1', E'9223372036854775807', E'Partner 1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
NOTICE:  NEW: (1,1,1,9223372036854775807,,"Partner 1")
┌ Info: SELECT "partnerrevisions"."id" AS "partnerrevisions_id", "partnerrevisions"."ref_component" AS "partnerrevisions_ref_component", "partnerrevisions"."ref_validfrom" AS "partnerrevisions_ref_validfrom", "partnerrevisions"."ref_invalidfrom" AS "partnerrevisions_ref_invalidfrom", "partnerrevisions"."description" AS "partnerrevisions_description" FROM "partnerrevisions" WHERE "id" = 1 ORDER BY partnerrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_version = E'1' AND is_committed = 0 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validi

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" WHERE ref_history = E'1' AND tsrdb @> TIMESTAMPTZ E'2038-01-18T03:14:06.999+00:00' AND tsrworld && tstzrange(E'2014-05-31T20:00:01.001+00:00',E'2038-01-19T03:14:06.999+00:00') AND is_committed=1 ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: UPDATE validityintervals

┌ Info: UPDATE workflows SET  "id" = E'1', "ref_history" = E'1', "ref_version" = E'1', "tsdb_validfrom" = E'2022-07-07T21:15:20.072+01:00', "tsw_validfrom" = E'2014-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '1' RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 1 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


In [4]:
find(Partner)

┌ Info: SELECT "partners"."id" AS "partners_id", "partners"."ref_history" AS "partners_ref_history", "partners"."ref_version" AS "partners_ref_version" FROM "partners" ORDER BY partners.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


1-element Vector{Partner}:
 Partner
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 1     |
| ref_history::DbId | 1     |
| ref_version::DbId | 1     |


Create tariffs

In [30]:
# create Tariffs
function create_tariff(dsc)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
end


create_tariff("Life Risk Insurance")
create_tariff("Terminal Illness")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 14 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'14' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 14 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/Se

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 15 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'15' ) RETURNING id
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions"

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffrevisions"."id" AS "tariffrevisions_id", "tariffrevisions"."ref_component" AS "tariffrevisions_ref_component", "tariffrevisions"."ref_validfrom" AS "tariffrevisions_ref_validfrom", "tariffrevisions"."ref_invalidfrom" AS "tariffrevisions_ref_invalidfrom", "tariffrevisions"."description" AS "tariffrevisions_description" FROM "tariffrevisions" ORDER BY tariffrevisions.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


2-element Vector{TariffRevision}:
 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Life Risk Insurance |
| id::DbId              | 12                  |
| ref_component::DbId   | 12                  |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 14                  |

 TariffRevision
| KEY                   | VALUE               |
|-----------------------|---------------------|
| description::String   | Terminal Illness    |
| id::DbId              | 13                  |
| ref_component::DbId   | 13                  |
| ref_invalidfrom::DbId | 9223372036854775807 |
| ref_validfrom::DbId   | 15                  |


In [29]:


map(find(Tariff)) do t
    SearchLight.delete(t)
end

find(Tariff)

#   t2 = Tariff()
#   tr2 = TariffRevision(description="Terminal Illness")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#   create_entity!(w0)
#   create_component!(t2, tr2, w0)
#   commit_workflow!(w0)
#
#   t3 = Tariff()
#   tr3 = TariffRevision(description="Occupational Disability")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#
#   create_entity!(w0)
#   create_component!(t3, tr3, w0)
#   commit_workflow!(w0)
#
#   t4 = Tariff()
#   tr4 = TariffRevision(description="Profit participation")
#   w0 = Workflow(
#       tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
#   )
#   create_entity!(w0)
#   create_component!(t4, tr4, w0)
#   commit_workflow!(w0)


┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: DELETE FROM tariffs WHERE id = '10'
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: DELETE FROM tariffs WHERE id = '11'
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/mf/.julia/packages/SearchLightPostgreSQL/TgpHy/src/SearchLightPostgreSQL.jl:134


Tariff[]